In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
# usual imports
import os
import sys
import numpy as np
from rail.core.utils import RAILDIR
#from rail.estimation.algos.sompz_version.utils import RAIL_SOMPZ_DIR
#from rail.pipelines.estimation.inform_all import InformPipeline
from rail.core import common_params
import matplotlib.pyplot as plt

In [3]:
# use local repo with local edits
# rail_sompz_repo_develop_list = ["/Users/jmyles/repositories/rail_sompz",
#                                 "/Users/jmyles/repositories/rail_sompz/lib",
#                                 "/Users/jmyles/repositories/rail_sompz/lib/python3.10",
#                                 "/Users/jmyles/repositories/rail_sompz/lib/python3.10/site-packages"]
# for rail_sompz_repo_develop in rail_sompz_repo_develop_list:
#     sys.path.insert(0, rail_sompz_repo_develop)
from rail.estimation.algos.sompz import SOMPZInformer
from rail.estimation.algos.sompz import SOMPZEstimator

In [4]:
from rail.core.data import TableHandle
from rail.core.stage import RailStage

In [5]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [6]:
from rail.core.utils import find_rail_file
#trainFile = find_rail_file('examples_data/testdata/test_dc2_training_9816.hdf5')
testFile = find_rail_file('examples_data/testdata/test_dc2_validation_9816.hdf5')
#deep_data = DS.read_file("training_data", TableHandle, trainFile)
#wide_data = DS.read_file("input_wide_data", TableHandle, testFile)

In [7]:
#datadir = '/global/cfs/projectdirs/desc-pz/users/jmyles/sompz_desc/' 
datadir = '/Users/jmyles/data/sompz_desc'
hsc_file = os.path.join(datadir, "hsc_deep_incl_nir_incl_specz_TEST_ONLY_NO_SCIENCE_2023-10-27.fits")
spec_data = DS.read_file("training_data", TableHandle, hsc_file)
balrog_data = DS.read_file("training_data", TableHandle, hsc_file)
wide_data = DS.read_file("training_data", TableHandle, hsc_file)

In [ ]:
balrog_data['overlap_weight'] = np.ones(len(balrog_data))
wide_data['overlap_weight'] = np.ones(len(balrog_data))

balrog_data['cell_deep'] = None
balrog_data['cell_wide_unsheared'] = None
wide_data['cell_wide_unsheared'] = None

In [8]:
subset = deep_data()[::10]

In [11]:
smaller_data = DS.add_data("smaller_data", subset, TableHandle)

In [12]:
bands = ['u','g','r','i','z','y','J','H', 'K']

deepbands = []
deeperrs = []
zeropts = []
for band in bands:
    deepbands.append(f'flux_DEEP_{band}')
    deeperrs.append(f'flux_err_DEEP_{band}')
    zeropts.append(30.)

widebands = []
wideerrs = []  
for band in bands[1:5]:
    widebands.append(f'{band}_cmodel_flux')
    wideerrs.append(f'{band}_cmodel_fluxerr')
    
refband_deep=deepbands[3]
refband_wide=widebands[2]

In [13]:
som_params = dict(inputs_deep=deepbands, input_errs_deep=deeperrs, 
                  zero_points_deep=zeropts, 
                  convert_to_flux_deep=False, convert_to_flux_wide=False, 
                  set_threshold_deep=True, thresh_val_deep=1.e-5, 
                  som_shape_wide=(25, 25), som_minerror_wide=0.005,
                  som_take_log_wide=False, som_wrap_wide=False)

In [14]:
som_estimate = SOMPZEstimator.make_stage(name="som_estimator", 
                                         data_path=testFile,
                                         deep_bands=deepbands,
                                         err_deep_bands=deeperrs,  
                                         ref_band_deep=refband_deep,
                                         wide_bands=widebands,
                                         err_wide_bands=wideerrs,    
                                         ref_band_wide=refband_wide,
                                         deep_groupname='', 
                                         hdf5_groupname='',
                                         model="TEST_HSC_model_2023-10-27b.pkl", 
                                         **som_params)

Inserting handle into data store.  model: TEST_HSC_model_2023-10-27b.pkl, som_estimator
initialized model {'deep_som': <rail.estimation.algos.som.NoiseSOM object at 0x15db810f0>, 'wide_som': <rail.estimation.algos.som.NoiseSOM object at 0x15a4b85b0>, 'deep_columns': ['flux_DEEP_u', 'flux_DEEP_g', 'flux_DEEP_r', 'flux_DEEP_i', 'flux_DEEP_z', 'flux_DEEP_y', 'flux_DEEP_J', 'flux_DEEP_H', 'flux_DEEP_K'], 'deep_err_columns': ['flux_err_DEEP_u', 'flux_err_DEEP_g', 'flux_err_DEEP_r', 'flux_err_DEEP_i', 'flux_err_DEEP_z', 'flux_err_DEEP_y', 'flux_err_DEEP_J', 'flux_err_DEEP_H', 'flux_err_DEEP_K'], 'wide_columns': ['g_cmodel_flux', 'r_cmodel_flux', 'i_cmodel_flux', 'z_cmodel_flux'], 'wide_err_columns': ['g_cmodel_fluxerr', 'r_cmodel_fluxerr', 'i_cmodel_fluxerr', 'z_cmodel_fluxerr']}


In [15]:
nsamp = 100
deep_fluxes_as_ndarray    = np.array(subset[deepbands], copy=False).view((np.float32, len(subset[deepbands].dtype.names)))[:nsamp]
deep_flux_errs_as_ndarray = np.array(subset[deeperrs] , copy=False).view((np.float32, len(subset[deepbands].dtype.names)))[:nsamp]
wide_fluxes_as_ndarray    = np.array(subset[widebands], copy=False).view((np.float32, len(subset[widebands].dtype.names)))[:nsamp]
wide_flux_errs_as_ndarray = np.array(subset[wideerrs] , copy=False).view((np.float32, len(subset[widebands].dtype.names)))[:nsamp]

In [16]:
som_estimate.estimate(deep_fluxes_as_ndarray,
                      deep_flux_errs_as_ndarray,
                      wide_fluxes_as_ndarray,
                      wide_flux_errs_as_ndarray,)

classifying 0
classifying 10
classifying 20
classifying 30
classifying 40


/Users/jmyles/repositories/rail_sompz/src/rail/estimation/algos/som.py:360: RuntimeWarning: overflow encountered in exp
  w = np.minimum(np.exp(2 * (vf - 4)), 1000.)


classifying 50
classifying 60
classifying 70
classifying 80
classifying 90
classifying 0
classifying 10
classifying 20
classifying 30
classifying 40
classifying 50
classifying 60
classifying 70
classifying 80
classifying 90
Inserting handle into data store.  model: TEST_HSC_model_2023-10-27b.pkl, som_estimator
> /Users/jmyles/repositories/rail_sompz/src/rail/estimation/algos/sompz.py(264)_estimate_pdf()
    262         pdb.set_trace()
    263         # TODO: compute p(z|c), redshift distributions in deep SOM cells
--> 264         pz_c = None
    265         print('hello, _estimate_pdf')
    266 



ipdb>  q
